### Visualization of the current learned Model


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%reload_ext autoreload

In [ ]:
from __future__ import print_function

import sys
import numpy as np
import matplotlib.pyplot as plt
sys.path.insert(0,'../../../../')
from DeepCrazyhouse.src.runtime.color_logger import enable_color_logging
from DeepCrazyhouse.src.domain.util import *
from DeepCrazyhouse.src.domain.variants.input_representation import planes_to_board, board_to_planes
from DeepCrazyhouse.src.tools.visualization.plane_representation import get_plane_vis
from DeepCrazyhouse.src.domain.variants.output_representation import *
from DeepCrazyhouse.src.preprocessing.dataset_loader import load_pgn_dataset
from DeepCrazyhouse.src.training.train_cli_util import *
from DeepCrazyhouse.src.training.trainer_agent_pytorch import load_torch_state
from DeepCrazyhouse.configs.main_config import main_config
from glob import glob
import re
import logging
from matplotlib import rc
%matplotlib inline

In [ ]:
#plt.style.use("seaborn-whitegrid")
enable_color_logging()
plt.style.use("seaborn-white")
#plt.style.use("seaborn-colorblind")
#plt.style.use('seaborn-notebook')
plt.rcParams['legend.frameon'] = 'True'
plt.rcParams['legend.framealpha'] = 1.0
#plt.rcParams['grid.alpha'] = 0.0
export_plots = True #True
#sns.set()
#sns.set_style("whitegrid")
#colors = sns.color_palette("colorblind")
rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
## for Palatino and other serif fonts use:
rc('font',**{'family':'serif','serif':['Palatino']})
rc('text', usetex=True)
#print(plt.rcParams) # to examine all values
fac = 0.7
print(plt.rcParams.get('figure.figsize'))
#plt.rcParams['figure.figsize'] = [8.0*fac, 5.5*fac]

logging.getLogger().setLevel(logging.INFO)

In [ ]:
def apply_def(filename):
    #plt.grid(axis='x')
    #plt.xlabel('Number of Training Samples processed')
    if export_plots is True:
        plt.savefig('./plots/update/%s.png'%filename, bbox_inches='tight') #, pad_inches = 0)
        plt.savefig('./plots/update/%s.pdf'%filename, bbox_inches='tight') #, pad_inches = 0)
    #matplotlib2tikz.save('./plots/update/tikz/%s.tex' % filename)
    plt.show()

In [ ]:
main_config

In [ ]:
# dimensions of the generated pictures for each filter.
#input_shape = (1, 34, 8, 8)
input_shape = (52, 8, 8)
layer_idx = 25
export_plots = True #False
export_tikz = False #True
latex_style = True
data_name = 'data'
normalize = True
activation =  'relu' #'leakyrelu' #'leakyrelu' # 'prelu' # 
#img_idx = 10
img_idx = 42
#img_idx = 60
opts = 10

if latex_style:
    from matplotlib import rc
    plt.rc('text', usetex=True)
    plt.rc('font', family='serif')

In [ ]:
train_config = TrainConfig()


In [ ]:
train_config.tar_file

In [ ]:
train_config.model_type

In [ ]:
model = create_pytorch_model(input_shape, train_config)

### Load model weights

In [ ]:
if train_config.tar_file != "":
    print("load model weights")
    load_torch_state(model, torch.optim.SGD(model.parameters(), lr=train_config.max_lr), Path(train_config.tar_file),
                     train_config.device_id)

In [ ]:
model.eval()

In [ ]:
# Hook setup
activations = {}
def get_activation(name):
    def hook(model, input, output):
        activations[name] = output.detach()
    return hook

model.body_spatial[0].body[0].register_forward_hook(get_activation('stem_conv0'))
for i in range(len(model.res_blocks)):
    try:
        model.res_blocks[i].body[0].register_forward_hook(get_activation(f'res_{i}_conv0'))
    except:
        model.res_blocks[i].patch_embed.register_forward_hook(get_activation(f'res_{i}_patch_embed'))

model.policy_head.body[3].register_forward_hook(get_activation('policy_conv1'))
model.value_head.body[0].register_forward_hook(get_activation('value_conv0'))

In [ ]:
len(model.res_blocks)

In [ ]:
board = chess.Board(fen="2r2r1k/1b5p/p2Q1q2/3p1P2/3Pp3/8/6RP/6RK w - - 0 1")

In [ ]:
board

In [ ]:
x = board_to_planes(board, normalize=normalize, mode=2)
#pred = predict_single(net, x_start_pos, tc.select_policy_from_plane)

In [ ]:
# Run the model
with torch.no_grad():
    output = model(torch.Tensor(np.expand_dims(x, axis=0))) #.to(ctx))

In [ ]:
output

In [ ]:
from DeepCrazyhouse.src.domain.variants.plane_policy_representation import FLAT_PLANE_IDX
from DeepCrazyhouse.src.domain.variants.output_representation import policy_to_moves, policy_to_best_move, policy_to_move

In [ ]:
output[1][0].shape

In [ ]:
output[1][0].softmax(dim=0)

In [ ]:
def predict_single(net, x, select_policy_from_plane=True):
    
    out = [None, None]

    with torch.no_grad():
        pred = net(torch.Tensor(np.expand_dims(x, axis=0))) #.to(ctx))
        out[0] = pred[0].to(torch.device("cpu")).numpy()
        out[1] = pred[1].to(torch.device("cpu")).softmax(dim=1).numpy()
    if select_policy_from_plane:
        out[1] = out[1][:, FLAT_PLANE_IDX]
    
    return out

In [ ]:
out = predict_single(model, x)
selected_moves, probs = policy_to_moves(board, out[1][0])

In [ ]:
probs[0]

In [ ]:
selected_moves

In [ ]:
move_probs = {}
for move, prob in zip(selected_moves, probs):
    move_probs[str(move)] = prob

In [ ]:
move_probs

In [ ]:
readable_output = {}
readable_output['value_output'] = output[0]
readable_output['policy_output'] = move_probs
readable_output['loss_draw_win_output'] = output[3].softmax(dim=1)
readable_output['plys_to_end_output'] = output[4]

In [ ]:
# Visualization function for activations
def plot_activations(layer, num_cols=4, num_activations=16):
    num_kernels = layer.shape[1]
    fig, axes = plt.subplots(nrows=(num_activations + num_cols - 1) // num_cols, ncols=num_cols, figsize=(12, 12))
    for i, ax in enumerate(axes.flat):
        if i < num_kernels:
            ax.imshow(layer[0, i].cpu().numpy(), cmap='twilight')
            ax.axis('off')
    plt.tight_layout()
    #plt.show()

In [ ]:
def plot_x(x, num_cols=8, num_activations=52):
    num_kernels = x.shape[0]
    fig, axes = plt.subplots(nrows=(num_activations + num_cols - 1) // num_cols, ncols=num_cols, figsize=(12, 12))
    for i, ax in enumerate(axes.flat):
        if i < num_kernels:
            ax.imshow(x[i], cmap='Blues')
            ax.axis('off')
    plt.tight_layout()

In [ ]:
plot_x(x)

In [ ]:
# Display a subset of activations
#plot_activations(activations['res_0_conv0'])#, num_cols=8, num_activations=64)
plot_activations(activations['stem_conv0'], num_cols=16, num_activations=224)
apply_def('stem_conv0')

In [ ]:
plot_activations(activations['res_0_conv0'], num_cols=16, num_activations=224)
apply_def('res_0_conv0')

In [ ]:
plot_activations(activations['res_1_conv0'], num_cols=16, num_activations=224)


In [ ]:
plot_activations(activations['res_35_conv0'], num_cols=16, num_activations=224)
apply_def('res_36_conv0')

In [ ]:
plot_activations(activations['res_36_patch_embed'], num_cols=16, num_activations=224)

In [ ]:
plot_activations(activations['policy_conv1'], num_cols=8, num_activations=50)
apply_def('policy_conv1')

In [ ]:
plot_activations(activations['value_conv0'], num_cols=4, num_activations=8)
apply_def('value_conv0')

In [ ]:
sum_act = activations['policy_conv1'][0][0]

In [ ]:
for i in range(1,activations['policy_conv1'].shape[1]):
    sum_act += activations['policy_conv1'][0][i]

In [ ]:
plt.imshow(sum_act.cpu().numpy(), cmap='twilight')
plt.colorbar()
apply_def('sum_act_policy')

In [ ]:
sum_act_value = activations['value_conv0'][0][0]
for i in range(1,activations['value_conv0'].shape[1]):
    sum_act_value += activations['value_conv0'][0][i]

In [ ]:
plt.imshow(sum_act_value.cpu().numpy(), cmap='twilight')
plt.colorbar()
apply_def('sum_act_value')

### Export everything in a loop

In [ ]:
import pandas as pd
import pickle
df = pd.read_csv('dataset.csv')

In [ ]:
for idx, fen in enumerate(df['fen']):
    print(idx, fen)
    
    board = chess.Board(fen=fen)
    x = board_to_planes(board, normalize=normalize, mode=2)
    
    with torch.no_grad():
        output = model(torch.Tensor(np.expand_dims(x, axis=0)))
    
    prefix = f'data/alpha_vile_large/'
    #prefix = f'data/random/'

    out = predict_single(model, x)
    selected_moves, probs = policy_to_moves(board, out[1][0])

    move_probs = {}
    for move, prob in zip(selected_moves, probs):
        move_probs[str(move)] = prob
    
    readable_output = {}
    readable_output['value_output'] = output[0]
    readable_output['policy_output'] = move_probs
    readable_output['loss_draw_win_output'] = output[3].softmax(dim=1)
    readable_output['plys_to_end_output'] = output[4]

    input_data = {}
    input_data['x'] = x
    input_data['fen'] = fen
    
    with open(f'{prefix}{idx}_activations.pickle', 'wb') as handle:
        pickle.dump(activations, handle, protocol=pickle.HIGHEST_PROTOCOL)

    with open(f'{prefix}{idx}_output.pickle', 'wb') as handle:
        pickle.dump(output, handle, protocol=pickle.HIGHEST_PROTOCOL)

    with open(f'{prefix}{idx}_readable_output.pickle', 'wb') as handle:
        pickle.dump(readable_output, handle, protocol=pickle.HIGHEST_PROTOCOL)

    with open(f'{prefix}{idx}_input.pickle', 'wb') as handle:
        pickle.dump(input_data, handle, protocol=pickle.HIGHEST_PROTOCOL)